# フックを使用した Strands エージェントと AgentCore Memory チュートリアル

## 概要

このチュートリアルでは、フックを介して AgentCore Memory と統合された Strands エージェントを使用して、インテリジェントなパーソナルアシスタントを構築する方法を示します。エージェントは会話コンテキストを維持し、やり取りから学習してパーソナライズされた応答を提供します。

## チュートリアルの詳細

**ユースケース**: 数学アシスタント

| 項目              | 詳細                                                                             |
|:------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 長期会話型                                                                       |
| エージェントタイプ  | 数学アシスタント                                                                 |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore Summary Strategy for Memory、Memory の保存と取得のためのフック    |
| 例の複雑さ         | 中級                                                                             |


学習内容:
- 会話要約を使用した AgentCore Memory のセットアップ
- 自動保存と取得のためのメモリフックの作成
- 永続的なメモリを持つ Strands エージェントの構築
- 会話間でのメモリ機能のテスト

### シナリオコンテキスト

この例では、以前の会話の要約を保存する数学アシスタントの例を作成します。
この例の主な機能:
- **自動メモリ保存**: 会話が自動的に保存される
- **コンテキスト取得**: 以前の会話が現在の応答に反映される
- **要約生成**: 重要な情報が抽出されて要約される
- **ツール統合**: 数学演算のための電卓ツール

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 前提条件

このチュートリアルを実行するには以下が必要です:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 権限を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

## ステップ 1: 環境セットアップ
このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義することから始めましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [ ]:
import os
import logging
from strands import Agent
from datetime import datetime
from strands_tools import calculator
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - replace with your values
REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"

## ステップ 2: メモリリソースの作成

このステップでは、要約戦略を使用してメモリリソースを作成します。このリソースは会話データを保存して整理します。定義する戦略は、会話の要約を自動的に生成し、整理されたネームスペースに保存します。

まず、数学アシスタント用のカスタムプロンプトを作成しましょう。

In [ ]:
# 数学アシスタント用のカスタムプロンプト
CUSTOM_PROMPT = """
あなたのタスクは、ユーザーの会話から数学学習データを抽出することです。ユーザーの進捗をメモリシステムに保存し、彼らの数学レベルを理解して進歩を支援します。

あなたは会話を分析してユーザーの数学学習パターンを抽出する任務を与えられています。2 つのデータセットを分析します:

<past_conversation>
[コンテキストのために、ユーザーと数学家庭教師の過去の会話がここに配置されます]
</past_conversation>

<current_conversation>
[ユーザーと数学家庭教師の現在の会話がここに配置されます]
</current_conversation>

あなたの仕事は、ユーザーの数学学習プロファイルを特定して分類することです:
- 正解/不正解の問題からユーザーの現在の数学レベルを抽出
- 質問の仕方と説明への反応からユーザーの好む学習スタイルを抽出
- パフォーマンスパターンからトピックの強みと弱点を抽出
- 学習の進捗を追跡し、強化が必要な領域を特定
"""

In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "MathAssistant"
# Define memory strategy for conversation summaries
strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}"],
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "global.anthropic.claude-haiku-4-5-20251001-v1:0",
                        "appendToPrompt": CUSTOM_PROMPT
                    }
                },
    }}}
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies, # Use the defined long term strategies
        description="Memory for tutorial agent",
        event_expiry_days=30,
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## ステップ 3: メモリフックプロバイダーの作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリフックは 2 つの主要な機能を提供します:

1. **メモリの取得**: ユーザーがメッセージを送信したときに、関連する過去の会話を自動的に取得
2. **メモリの保存**: エージェントが応答した後に新しい会話を保存

これにより、手動管理なしでシームレスなメモリ体験が実現されます。

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
    
    def retrieve_memories(self, event: MessageAddedEvent):
        """Retrieve relevant memories before processing user message"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")
            
            try:
                # Get actor_id from agent state
                actor_id = event.agent.state.get("actor_id")
                if not actor_id:
                    logger.warning("Missing actor_id in agent state")
                    return
                
                namespace = f"/students/math/{actor_id}"
                
                # Retrieve relevant memories
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=namespace,
                    query=user_message
                )
                
                # Extract memory content
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

## ステップ 4: メモリ付きエージェントの作成

ここでは、Strands エージェントを作成し、メモリフックプロバイダーと接続します。このエージェントには 2 つの主要な機能があります:

1. **メモリ統合**: 作成したメモリフックが自動コンテキスト取得を可能にする
2. **電卓ツール**: 必要に応じてエージェントが数学演算を実行できる

この組み合わせにより、過去のやり取りを記憶し、便利な計算を実行できるパーソナルアシスタントが作成されます。

In [ ]:
# Create memory hook provider
memory_hooks = MemoryHookProvider(memory_id, client)

# Create agent with memory hooks and calculator tool
agent = Agent(
    hooks=[memory_hooks],
    model = "global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[calculator],
    system_prompt="You are a helpful personal math tutor. You assist users in solving math problems and provide personalized assistance.",
    state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
)

print("✅ メモリフック付きエージェントが作成されました。")

**エージェントのセットアップ完了! テストしてみましょう。**

## メモリ機能のテスト

このセクションでは、一連のやり取りを通じてエージェントのメモリ機能をテストします。エージェントが時間の経過とともにどのようにコンテキストを構築し、以前のやり取りを思い出すかを観察します。

まず、自己紹介をして数学の質問をしてみましょう:

In [ ]:
# First interaction - introduce yourself
response1 = agent("Hi, I'm John and I just enrolled in Discrete Math course. Help me solve this: How many ways can I arrange 5 books on a shelf?")
print(f"エージェント: {response1}")

エージェントに別の計算タスクを与えてみましょう:

In [ ]:
# Second interaction - another calculation
response2 = agent("I learn better with step-by-step explanation with example questions. Can you explain modular arithmetic? What's 17 mod 5?")
print(f"エージェント: {response2}")

次に、エージェントが私たちのことを覚えているかどうかを確認しましょう。

**注意:** メモリが抽出、統合、保存されるまで約 20 秒待ってください。

In [ ]:
# Third interaction - test memory recall
response3 = agent("I got that right! What's the immediate next step that I should study after modular arithmetic?")
print(f"エージェント: {response3}")

最後に、エージェントが計算履歴を覚えているかどうかを確認しましょう:

In [ ]:
# Fourth interaction - test context awareness
response4 = agent("This is too hard, can we try something easier?")
print(f"エージェント: {response4}")

### メモリの保存確認

最後のステップとして、会話が AgentCore Memory に適切に保存されていることを確認します。これにより、メモリフックが正しく動作し、エージェントが将来のやり取りでこの情報にアクセスできることが実証されます。

In [ ]:
# Check stored memories
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"/students/math/{ACTOR_ID}",
        query="mathematics calculations"
    )
    
    print(f"\n📚 {len(memories)} 件のメモリが見つかりました:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"メモリの取得エラー: {e}")

チュートリアル完了!

主な学び:
- メモリフックが会話コンテキストを自動的に保存および取得
- エージェントは複数のやり取りにわたって状態を維持可能
- AgentCore Memory は関連するコンテキストのセマンティック検索を提供
- ツールをメモリと組み合わせて機能を強化可能

## クリーンアップ

### オプション: メモリリソースの削除

チュートリアル完了後、不要なコストを避けるためにメモリリソースを削除することをお勧めします。以下のコードはクリーンアップ用に提供されていますが、デフォルトではコメントアウトされています。

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")